## 建模思路

类比语言模型的建立，可以建立一个物体共现模型：

* 将一幅正常图像中所有共同出现的物体定义为集合I。那么I中任意一个元素，将会与I中其它元素经常出现在同一个图像中。和语言模型中词汇在上下文中出现的概率类似。那么我们可以训练一个神经网络模型M，表达I中各个元素共同出现的概率。

* 当给定一个待判定集合S，使用模型M预测S中所有元素的出现概率，将预测结果中概率最小的元素，判定为不和谐元素。



## 训练数据概览

* 基于MSCOCO数据集 train2017 中 instance detecion 数据

* 包含物体类别总共 80 个

* 共有图片118287个

* 每幅图像中包含物体类别的数量分布：

		(0, 3]      84213
		(3, 6]      27039
		(6, 9]       5165
		(9, 12]       774
		(12, 15]       72
		(15, 18]        3
		(18, 21]        0
		(21, 24]        0
		(24, 27]        0


In [1]:
# coding=utf-8

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from itertools import compress
import time
import os

In [2]:
PAD_IDX = 0    # 用于填充的分类索引
UNK_IDX = 91   # 不在coco已知分类中的物体索引

VOCAB_SIZE = 92  # 所有分类的个数
EMBEDDING_SIZE = 256  
CONTEXT_SIZE = 16  # 每幅图片中物体类别数

In [3]:
cocoCatIds = \
[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 27,
 28,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 67,
 70,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 84,
 85,
 86,
 87,
 88,
 89,
 90]

### 因为coco的category id是不连续的，为了防止中间不存在的id干扰loss计算，设置掩码把它们排除掉

In [4]:
mask = np.zeros(shape=(VOCAB_SIZE,), dtype=np.float32)
for catId in cocoCatIds:
    mask[catId] = 1.
mask[PAD_IDX] = 1.
mask[UNK_IDX] = 1.

In [5]:
len(mask)

92

## 建立模型

* 词表的建立：
  * 直接以coco category id作为词表索引，但是这个id是不连续的，对于不在category id集合中的索引，通过mask将其中计算中屏蔽掉。
  * PAD索引为0。
  * UNK索引为91。

* 输入层：每幅图片中物体类别数（CONTEXT_SIZE）。

* embedding层：92 * 256，这里设置词嵌入长度为256。

* 全联接隐层：256 * 256，tanh为激活函数。

* 输出层：256 * 92，输出为词表大小的向量，sigmoid为激活函数（因为输出表示该类别出现的概率），不需要计算softmax。


In [6]:
tf.reset_default_graph()

In [7]:
learning_rate = 0.01

In [8]:
inputs_ = tf.placeholder(tf.int32, shape=[None, CONTEXT_SIZE])
labels_ = tf.placeholder(tf.float32, shape=[None, VOCAB_SIZE])

In [9]:
# embedding层
embeddings = tf.Variable(tf.random_uniform([VOCAB_SIZE, EMBEDDING_SIZE]))
input_embeddings = tf.nn.embedding_lookup(embeddings, inputs_)
context_embeddings = tf.reduce_sum(input_embeddings, axis=1)

In [10]:
# hidden层
hidden_output = tf.layers.dense(context_embeddings, EMBEDDING_SIZE, activation=tf.nn.tanh)

# output层
raw_output = tf.layers.dense(hidden_output, VOCAB_SIZE)
final_output = tf.sigmoid(raw_output)

In [11]:
final_output

<tf.Tensor 'Sigmoid:0' shape=(?, 92) dtype=float32>

In [12]:
# loss and GD optimizer
loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=raw_output, labels=labels_)*mask
    # tf.nn.sparse_softmax_cross_entropy_with_logits(logits=raw_output, labels=labels_) * mask
)

train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

saver = tf.train.Saver()

In [13]:
init = tf.global_variables_initializer()

### 训练数据预处理

In [14]:
# 将数据从输入为单个物体类别，转换为图片包含的所有物体类别，实际就是增加了PADDING。将原来的N*1维数组，变成了N*CONTEXT_SIZE维数组

def prepare_data(raw_data):
    data = np.zeros(shape=(len(raw_data), CONTEXT_SIZE), dtype=np.int32)
    for i in range(len(raw_data)):
        for j in range(len(raw_data[i])):
            if j == 16:
                break;
            data[i][j] = raw_data[i][j]
    return data

def prepare_labels(raw_labels):
    labels = np.zeros(shape=(len(raw_labels), VOCAB_SIZE), dtype=np.float32)
    for i in range(len(raw_labels)):
        for item in raw_labels[i]:
            labels[i][item] = 1.
    return labels

In [15]:
train_rdata = np.load("data/train_data.npy")
train_data = prepare_data(train_rdata)
train_rlabels = np.load("data/train_labels.npy")
train_labels = prepare_labels(train_rlabels)

### 对模型进行训练

In [16]:
session = tf.Session()
session.run(init)
print('Initialized')

batch_size = 32
epochs = 1
average_loss = 0
step = 0
eval_interval = 1000
start_time = time.time()
try:
    for epoch in range(epochs):
        num_inputs = len(train_data)
        order = np.arange(num_inputs)
        np.random.shuffle(order)
        for j in range(0, num_inputs, batch_size):
            step += 1
            batch_index = order[j: j + batch_size]
            batch_inputs = train_data[batch_index]
            batch_labels = train_labels[batch_index]
            feed_dict = {inputs_: batch_inputs, \
                         labels_: batch_labels}
            # We perform one update step by evaluating the optimizer op (including it
            # in the list of returned values for session.run()
            _, loss_val = session.run([train_step, loss], feed_dict=feed_dict)
            average_loss += loss_val
            # if step == 100:
            #     break

            if step % eval_interval == 0:
                average_loss /= eval_interval
                # The average loss is an estimate of the loss over the last 2000 batches.
                print('Average loss at epoch ', step, ': ', average_loss)
                average_loss = 0

except KeyboardInterrupt:
    print('Interrupted')
finally:
    end_time = time.time()
    print('\ntime: {:.2f} s'.format(end_time - start_time))
    
    

Initialized
Average loss at epoch  1000 :  0.189834122762
Average loss at epoch  2000 :  0.131263718776
Interrupted

time: 12.18 s


In [17]:
model_base_path = './models/'
model_name = 'instance-context-model'
saver.save(session, os.path.join(model_base_path, model_name))

'./models/instance-context-model'